In [6]:
from google.colab import drive
import os
import numpy as np
from tensorflow.keras.models import load_model

drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/spacecraft_anomaly_project'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
MODEL_DIR = os.path.join(BASE_DIR, 'models', 'autoencoder')

# Load data
X = np.load(os.path.join(PROCESSED_DIR, 'X_windows.npy'))
y = np.load(os.path.join(PROCESSED_DIR, 'y_windows.npy'))  # for evaluation

# Load model
model = load_model(os.path.join(MODEL_DIR, 'lstm_autoencoder.h5'))

print(f"X shape: {X.shape}, y shape: {y.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


X shape: (2094, 30, 21), y shape: (2094,)


In [7]:
# Predict reconstruction for all windows
X_pred = model.predict(X, verbose=1)

# Compute Mean Squared Error per window
mse = np.mean(np.mean((X - X_pred)**2, axis=2), axis=1)

print(f"Reconstruction error shape: {mse.shape}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step
Reconstruction error shape: (2094,)


In [8]:
# Choose threshold (95th percentile of training errors)
threshold = np.percentile(mse[y == 0], 95)
print(f"Anomaly detection threshold (95th percentile of normal data): {threshold:.6f}")

Anomaly detection threshold (95th percentile of normal data): 1.353446


In [9]:
# Flag anomaly if reconstruction error > threshold
y_pred = (mse > threshold).astype(int)

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

acc = accuracy_score(y, y_pred)
prec = precision_score(y, y_pred)
rec = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

print(f"🔍 Detection Metrics:\nAccuracy: {acc:.4f}\nPrecision: {prec:.4f}\nRecall: {rec:.4f}\nF1 Score: {f1:.4f}")

🔍 Detection Metrics:
Accuracy: 0.3295
Precision: 0.9477
Recall: 0.1990
F1 Score: 0.3289
